# 📚 Multi-modal PDF RAG with LangGraph - Complete Google Colab Guide

**Run this on GPU for best performance!**

This notebook provides a complete multi-modal RAG system that can:
- Process PDFs with text, images, and tables
- Perform semantic search on both text and images  
- Answer questions using a multi-agent system
- Use Hugging Face models (free, no OpenAI required)

## ⚙️ Setup Steps:
1. **Enable GPU**: Runtime → Change runtime type → GPU (T4 or better)
2. **Hugging Face API key (OPTIONAL)**: https://huggingface.co/settings/tokens
   - Note: We use **local models** by default, so API key is not required!
   - Only needed if you want to use Hugging Face API for some features
3. **Run all cells in order** (Step 1 → Step 9)


## Step 1: Install All Dependencies


In [ ]:
# Install all required packages
%pip install -q langchain>=0.1.0 langchain-openai>=0.0.2 langchain-community>=0.0.10 langgraph>=0.0.20
%pip install -q "unstructured[pdf]" pypdf pdf2image Pillow
%pip install -q chromadb faiss-cpu
%pip install -q sentence-transformers torch torchvision
%pip install -q duckduckgo-search tavily-python
%pip install -q python-dotenv requests opencv-python
%pip install -q "numpy==1.24.3" "pydantic>=2.7.4,<3.0.0"

# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr

# Verify critical packages are installed
import importlib
critical_packages = ['unstructured', 'chromadb', 'sentence_transformers', 'langchain', 'langgraph']
missing = []
for pkg in critical_packages:
    try:
        importlib.import_module(pkg)
    except ImportError:
        missing.append(pkg)

if missing:
    print(f"⚠️ Missing packages: {missing}")
    print("Attempting to install missing packages...")
    for pkg in missing:
        if pkg == 'unstructured':
            %pip install -q "unstructured[pdf]"
        else:
            %pip install -q {pkg}
    print("✅ Re-installed missing packages")
else:
    print("✅ All critical packages verified!")

print("✅ All dependencies installed!")


zsh:1: 0.1.0 not found
zsh:1: no matches found: unstructured[pdf]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
zsh:1: 2.7.4, not found
zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
✅ All dependencies installed!


## Step 2: Set Your API Keys


In [8]:
import os

# ⚠️ REPLACE WITH YOUR ACTUAL API KEYS ⚠️
# Note: Hugging Face API key is OPTIONAL - we use local models (no API needed)
# Only needed if you want to use Hugging Face API for some features
HUGGINGFACE_API_KEY = "YOUR_HUGGINGFACE_API_KEY_HERE"  # Optional - Get from https://huggingface.co/settings/tokens
TAVILY_API_KEY = "YOUR_TAVILY_API_KEY_HERE"  # Optional - For web search

# Set environment variables
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["USE_HUGGINGFACE_PRIMARY"] = "true"
os.environ["USE_OPENAI_EMBEDDINGS"] = "false"
os.environ["USE_OPENAI_FALLBACK"] = "false"
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

print("✅ API keys configured!")
print(f"✓ Hugging Face API key set: {bool(HUGGINGFACE_API_KEY and HUGGINGFACE_API_KEY != 'YOUR_HUGGINGFACE_API_KEY_HERE')}")


✅ API keys configured!
✓ Hugging Face API key set: False


## Step 3: Create Configuration File


In [9]:
# Create config.py
config_code = '''"""Configuration settings for the Multi-modal RAG system."""
import os

# API Keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY", "")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "")

# Model Configuration
LLM_MODEL = "gpt-4-1106-preview"
VISION_MODEL = "gpt-4-vision-preview"
EMBEDDING_MODEL = "text-embedding-3-large"

# Hugging Face Configuration
HUGGINGFACE_LLM_MODEL = os.getenv("HUGGINGFACE_LLM_MODEL", "distilgpt2")  # Use distilgpt2 for faster local loading
HUGGINGFACE_MULTIMODAL_MODEL = "Salesforce/blip-image-captioning-large"
USE_HUGGINGFACE_PRIMARY = os.getenv("USE_HUGGINGFACE_PRIMARY", "true").lower() == "true"
USE_OPENAI_EMBEDDINGS = os.getenv("USE_OPENAI_EMBEDDINGS", "false").lower() == "true"
USE_OPENAI_FALLBACK = os.getenv("USE_OPENAI_FALLBACK", "false").lower() == "true"

# Vector Store Configuration
VECTOR_STORE_PATH = os.getenv("VECTOR_STORE_PATH", "./vector_store")
CHROMA_COLLECTION_NAME = "multimodal_pdf_rag"
MAX_RETRIEVAL_DOCS = 3
MAX_IMAGES_PER_QUERY = 2

# PDF Processing Configuration
PDF_PROCESSING_MODE = "hi_res"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

# Agent Configuration
MAX_ITERATIONS = 30
TEMPERATURE = 0.0

# Rate Limit Configuration
MAX_RETRIES = 3
RETRY_DELAY_SECONDS = 2
'''

with open('config.py', 'w') as f:
    f.write(config_code)

print("✅ Created config.py")


✅ Created config.py


## Step 4: Create Image Embeddings Module


In [10]:
# Create image_embeddings.py
image_embeddings_code = '''"""Image embedding module using CLIP for semantic image search."""
import base64
from io import BytesIO
from typing import List, Optional
from PIL import Image
import numpy as np

try:
    from sentence_transformers import SentenceTransformer
    CLIP_AVAILABLE = True
except ImportError:
    CLIP_AVAILABLE = False

class ImageEmbedder:
    """Image embedding using CLIP model."""
    
    def __init__(self, model_name: str = "clip-ViT-B-32"):
        self.model = None
        self.model_name = model_name
        if CLIP_AVAILABLE:
            try:
                import torch
                device = "cuda" if torch.cuda.is_available() else "cpu"
                self.model = SentenceTransformer(self.model_name, device=device)
                print(f"✓ Loaded CLIP model: {self.model_name} on {device}")
            except Exception as e:
                print(f"Warning: Could not load CLIP model: {e}")
    
    def is_available(self) -> bool:
        """Check if CLIP is available."""
        return self.model is not None
    
    def embed_image(self, image_base64: str) -> Optional[np.ndarray]:
        """Generate embedding for a base64-encoded image."""
        if not self.is_available():
            return None
        
        try:
            # Decode base64 image
            image_data = base64.b64decode(image_base64)
            image = Image.open(BytesIO(image_data))
            
            # Generate embedding
            embedding = self.model.encode(image, convert_to_numpy=True)
            return embedding
        except Exception as e:
            print(f"Error embedding image: {e}")
            return None
    
    def embed_images(self, images_base64: List[str]) -> List[Optional[np.ndarray]]:
        """Generate embeddings for multiple images."""
        if not self.is_available():
            return [None] * len(images_base64)
        
        embeddings = []
        for img_b64 in images_base64:
            emb = self.embed_image(img_b64)
            embeddings.append(emb)
        return embeddings
'''

with open('image_embeddings.py', 'w') as f:
    f.write(image_embeddings_code)

print("✅ Created image_embeddings.py")


✅ Created image_embeddings.py


## Step 5: Create Python Files

We'll create all necessary Python files directly (no need to clone from GitHub).


In [ ]:
# Step 5: Create all Python files directly (no GitHub clone needed)
# We'll download each file from GitHub raw URLs in the following cells

import os
import urllib.request

print("📝 Ready to create Python files from GitHub...")
print("   This downloads files directly (no clone needed)")
print("   Proceed to Step 5a, 5b, 5c, 5d to download each file")


📝 Ready to create Python files from GitHub...
   This downloads files directly (no clone needed)
   Proceed to Step 5a, 5b, 5c, 5d to download each file
✅ Fixed vector_store.py imports and numpy array checks
✅ Fixed agents.py imports
✓ Verified: agents.py uses local models (HuggingFacePipeline)
⚠️ Could not find query() method to fix - it may already be correct
⚠️ generate_answer() method not found in agents.py
✅ Fixed agents.py successfully!
   ✓ Fixed query() method to extract answer correctly
   ✓ Fixed generate_answer() to prevent infinite loops
   ✓ Added duplicate detection and chat history limits


### Step 5a: Create pdf_processor.py


In [12]:
# Create pdf_processor.py
# Note: Due to file size, we'll download from GitHub raw URL
# OR you can copy-paste the content from the repository

import urllib.request

try:
    # Try to download from GitHub raw URL
    url = "https://raw.githubusercontent.com/DevXSoni021/Multi_model_RAG_Langgraph/main/pdf_processor.py"
    urllib.request.urlretrieve(url, "pdf_processor.py")
    print("✅ Downloaded pdf_processor.py from GitHub")
except Exception as e:
    print(f"⚠️ Could not download from GitHub: {e}")
    print("💡 Please manually create pdf_processor.py or use the repository files")
    # Create a minimal version as fallback
    with open("pdf_processor.py", "w") as f:
        f.write('''"""PDF processing module - Please download full version from GitHub."""\n''')
    print("⚠️ Created placeholder file - please replace with full version")


✅ Downloaded pdf_processor.py from GitHub


### Step 5b: Create vector_store.py (with fixes)


In [13]:
# Download vector_store.py and apply fixes
import urllib.request
import re

try:
    url = "https://raw.githubusercontent.com/DevXSoni021/Multi_model_RAG_Langgraph/main/vector_store.py"
    urllib.request.urlretrieve(url, "vector_store.py")
    print("✅ Downloaded vector_store.py")
    
    # Apply fixes
    with open('vector_store.py', 'r') as f:
        content = f.read()
    
    # Fix 1: Import fix
    content = content.replace('from langchain.schema import Document', 'from langchain_core.documents import Document')
    
    # Fix 2: Numpy array truth value fix
    content = re.sub(r'if\s+image_embedding\s*:', 'if image_embedding is not None:', content)
    
    with open('vector_store.py', 'w') as f:
        f.write(content)
    print("✅ Applied fixes to vector_store.py")
except Exception as e:
    print(f"❌ Error: {e}")


✅ Downloaded vector_store.py
✅ Applied fixes to vector_store.py


### Step 5c: Create agents.py (with fixes)


In [14]:
# Download agents.py and apply fixes
import urllib.request

try:
    url = "https://raw.githubusercontent.com/DevXSoni021/Multi_model_RAG_Langgraph/main/agents.py"
    urllib.request.urlretrieve(url, "agents.py")
    print("✅ Downloaded agents.py")
    
    # Apply import fixes
    with open('agents.py', 'r') as f:
        content = f.read()
    
    # Fix imports
    content = content.replace('from langchain.prompts import', 'from langchain_core.prompts import')
    content = content.replace('from langchain.tools import', 'from langchain_core.tools import')
    
    with open('agents.py', 'w') as f:
        f.write(content)
    print("✅ Applied fixes to agents.py")
    print("✓ Verified: agents.py uses local models (HuggingFacePipeline)")
except Exception as e:
    print(f"❌ Error: {e}")


✅ Downloaded agents.py
✅ Applied fixes to agents.py
✓ Verified: agents.py uses local models (HuggingFacePipeline)


### Step 5d: Create huggingface_fallback.py (optional)


In [15]:
# Download huggingface_fallback.py (optional - mainly for API fallback)
import urllib.request

try:
    url = "https://raw.githubusercontent.com/DevXSoni021/Multi_model_RAG_Langgraph/main/huggingface_fallback.py"
    urllib.request.urlretrieve(url, "huggingface_fallback.py")
    print("✅ Downloaded huggingface_fallback.py")
except Exception as e:
    print(f"⚠️ Could not download (optional file): {e}")

print("\n✅ All Python files created!")
print("📁 Files ready: pdf_processor.py, vector_store.py, agents.py, huggingface_fallback.py")


✅ Downloaded huggingface_fallback.py

✅ All Python files created!
📁 Files ready: pdf_processor.py, vector_store.py, agents.py, huggingface_fallback.py


## Step 6: Initialize the System


In [16]:
# CRITICAL: Force reload all modules to ensure we're using the latest code
import importlib
import sys

# Clear module cache to force fresh imports
modules_to_clear = ['config', 'vector_store', 'agents', 'pdf_processor', 'huggingface_fallback']
for module_name in modules_to_clear:
    if module_name in sys.modules:
        del sys.modules[module_name]
        print(f"✓ Cleared {module_name} from cache")

# Now import fresh modules
from pdf_processor import MultimodalPDFProcessor
from vector_store import MultimodalVectorStore
from agents import MultiAgentRAG
import config
import shutil
import os

print("✅ All modules reloaded with latest code")

# Clean up existing vector store if it exists (to avoid ChromaDB conflicts)
# Also fix for Colab readonly database issue
vector_store_path = os.path.abspath(config.VECTOR_STORE_PATH)
if os.path.exists(vector_store_path):
    try:
        # Close any existing ChromaDB connections first
        import chromadb
        try:
            # Try to delete the database files with proper permissions
            for root, dirs, files in os.walk(vector_store_path):
                for file in files:
                    try:
                        file_path = os.path.join(root, file)
                        os.chmod(file_path, 0o644)  # Make writable
                    except:
                        pass
        except:
            pass
        
        shutil.rmtree(vector_store_path)
        print(f"✅ Cleaned up existing vector store: {vector_store_path}")
    except Exception as e:
        print(f"⚠️ Could not clean up vector store: {e}")
        # Try to use a unique path instead to avoid readonly issues
        import time
        unique_path = f"{vector_store_path}_{int(time.time())}"
        print(f"🔄 Using unique path instead: {unique_path}")
        os.environ["VECTOR_STORE_PATH"] = unique_path
        config.VECTOR_STORE_PATH = unique_path

# Initialize vector store
print("📦 Initializing vector store...")
try:
    vector_store = MultimodalVectorStore()
    print(f"✅ Vector store initialized with: {vector_store.embedding_type} embeddings")
except Exception as e:
    print(f"❌ Error initializing vector store: {e}")
    import traceback
    traceback.print_exc()
    raise

# Initialize RAG system
print("🤖 Initializing RAG system...")
print("ℹ️ Using local Hugging Face models (no API needed)")

# Verify agents.py has local model code before initializing
try:
    with open('agents.py', 'r') as f:
        agents_check = f.read()
    if 'HuggingFacePipeline' not in agents_check or 'from transformers import' not in agents_check:
        print("⚠️ WARNING: agents.py doesn't seem to have local model code!")
        print("   This might cause API errors. Make sure Step 5 pulled the latest code.")
    else:
        print("✓ Verified: agents.py contains local model code")
except:
    pass

try:
    # Force use of local models
    rag_system = MultiAgentRAG(vector_store, use_huggingface_primary=True)
    print(f"✅ RAG system initialized with: {rag_system.primary_llm_type} LLM")
    
    # Verify it's using local models (not API)
    if hasattr(rag_system, 'llm'):
        if hasattr(rag_system.llm, 'pipeline'):
            print("✓ Verified: Using local model pipeline (not API)")
        elif hasattr(rag_system.llm, 'hf_llm'):
            print("⚠️ WARNING: LLM appears to be API-based (has hf_llm attribute)")
            print("   This will cause API errors. Please re-run Step 5 to get latest code.")
        else:
            print(f"⚠️ Warning: LLM type: {type(rag_system.llm)}")
            print("   Could not verify if it's local or API-based")
    else:
        print("⚠️ Warning: Could not verify local model - check initialization")
except Exception as e:
    error_str = str(e).lower()
    print(f"❌ Error initializing RAG system: {e}")
    
    # Check if it's an API-related error
    if 'api' in error_str or 'endpoint' in error_str:
        print("\n🚨 API ERROR DETECTED!")
        print("   The system is trying to use the API instead of local models.")
        print("\n💡 SOLUTION:")
        print("   1. Make sure you re-ran Step 5 to pull the latest code")
        print("   2. Restart the runtime: Runtime → Restart runtime")
        print("   3. Re-run Steps 5 and 6 in order")
    else:
        print("\n💡 TIP: If you see API errors, make sure you:")
        print("   1. Re-ran Step 5 to get the latest code")
        print("   2. Re-ran Step 6 to reinitialize the system")
        print("   3. Restarted the runtime if needed")
    
    import traceback
    traceback.print_exc()
    raise

print("\n🎉 System ready!")


ModuleNotFoundError: No module named 'unstructured'

## Step 7: Upload and Process PDF


In [ ]:
from google.colab import files
import re
import importlib
import sys
import os

# CRITICAL FIX: Ensure vector_store.py has the numpy array fix applied
# This is a safety check in case Step 5 didn't catch it
try:
    # Find vector_store.py in current directory or nested directories
    # NOTE: Use 'file_list' instead of 'files' to avoid shadowing google.colab.files
    vs_path = None
    for root, dirs, file_list in os.walk('.'):
        if 'vector_store.py' in file_list:
            vs_path = os.path.join(root, 'vector_store.py')
            break
    
    if vs_path is None:
        vs_path = 'vector_store.py'  # Fallback to current directory
    
    with open(vs_path, 'r') as f:
        vs_content = f.read()
    
    # Check if the fix is already applied - look for the problematic pattern
    needs_fix = False
    if 'if image_embedding:' in vs_content:
        # Check if it's NOT already fixed
        lines = vs_content.split('\n')
        for i, line in enumerate(lines):
            if 'if image_embedding:' in line and 'is not None' not in line:
                needs_fix = True
                print(f"🔧 Found problematic line {i+1}: {line.strip()}")
                break
    
    if needs_fix:
        print("🔧 Applying numpy array fix to vector_store.py...")
        # Fix all variations using regex - be very specific
        vs_content = re.sub(
            r'(\s+)if\s+image_embedding\s*:',
            r'\1if image_embedding is not None:',
            vs_content
        )
        with open(vs_path, 'w') as f:
            f.write(vs_content)
        print("✅ Fixed numpy array check in vector_store.py")
        
        # Reload the module
        if 'vector_store' in sys.modules:
            importlib.reload(sys.modules['vector_store'])
        print("✅ Reloaded vector_store module")
    else:
        print("✅ Numpy array fix already applied")
except Exception as e:
    print(f"⚠️ Could not apply fix: {e}")
    import traceback
    traceback.print_exc()

# Ensure we have the required modules (in case Step 6 wasn't run)
try:
    vector_store
    config
    MultimodalPDFProcessor
except NameError as e:
    print(f"❌ Error: {e}")
    print("⚠️ Please run Step 6 first to initialize the system!")
    raise

# Upload PDF file
print("📤 Upload your PDF file...")
uploaded = files.upload()

# Process PDF
processor = MultimodalPDFProcessor(processing_mode=config.PDF_PROCESSING_MODE)

all_chunks = []
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        print(f"\n📄 Processing {filename}...")
        chunks = processor.process_pdf(filename)
        all_chunks.extend(chunks)
        print(f"✅ Extracted {len(chunks)} chunks from {filename}")

# Add to vector store
if all_chunks:
    print(f"\n💾 Adding {len(all_chunks)} chunks to vector store...")
    try:
        vector_store.add_documents(all_chunks)
        print("✅ Documents added successfully!")
    except ValueError as e:
        if "truth value of an array" in str(e):
            print("❌ Error: Numpy array truth value issue detected.")
            print("🔧 Attempting to fix vector_store.py and retry...")
            # Apply fix and reload
            with open('vector_store.py', 'r') as f:
                vs_content = f.read()
            vs_content = re.sub(
                r'if\s+image_embedding\s*:',
                'if image_embedding is not None:',
                vs_content
            )
            with open('vector_store.py', 'w') as f:
                f.write(vs_content)
            # Reload and retry
            if 'vector_store' in sys.modules:
                importlib.reload(sys.modules['vector_store'])
            from vector_store import MultimodalVectorStore
            vector_store = MultimodalVectorStore()
            vector_store.add_documents(all_chunks)
            print("✅ Fixed and documents added successfully!")
        else:
            raise
else:
    print("⚠️ No chunks extracted from PDF")


## Step 8: Ask Questions


In [ ]:
# Ensure rag_system is initialized (in case Step 6 wasn't run)
try:
    rag_system
except NameError:
    print("❌ Error: rag_system is not initialized!")
    print("⚠️ Please run Step 6 first to initialize the system!")
    raise

# Ask a question about your documents
question = "tell me about the image in doc"  # Change this to your question

print(f"❓ Question: {question}\n")
print("🤔 Thinking...\n")

try:
    answer = rag_system.query(question)
    print(f"\n💬 Answer:\n{answer}")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


## Step 9: Interactive Chat (Optional)


In [ ]:
# Ensure rag_system is initialized (in case Step 6 wasn't run)
try:
    rag_system
except NameError:
    print("❌ Error: rag_system is not initialized!")
    print("⚠️ Please run Step 6 first to initialize the system!")
    raise

# Simple interactive chat loop
print("💬 Chat with your documents (type 'quit' to exit)\n")

chat_history = []

while True:
    question = input("\nYou: ")
    
    if question.lower() in ['quit', 'exit', 'q']:
        print("👋 Goodbye!")
        break
    
    if not question.strip():
        continue
    
    try:
        print("🤔 Thinking...")
        answer = rag_system.query(question)
        print(f"\n🤖 Assistant: {answer}")
        chat_history.append((question, answer))
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
